## Import Data

In [1]:
import pandas as pd

data_2017 = pd.read_csv("/content/Sales Transactions-2017.csv")
print(data_2017)
data_2018 = pd.read_csv("/content/Sales Transactions-2018.csv")
print(data_2018)
data_2019 = pd.read_csv("/content/Sales Transactions-2019.csv")
print(data_2019)


             Date    Voucher  ...        Disc  Voucher Amount
0        1/4/2017      Sal:1  ...         NaN       13,100.00
1        1/4/2017      Sal:1  ...         NaN             NaN
2        1/4/2017      Sal:2  ...         NaN       30,990.00
3        1/4/2017      Sal:2  ...         NaN             NaN
4        1/4/2017      Sal:2  ...         NaN             NaN
...           ...        ...  ...         ...             ...
47285  31/03/2018  Sal:10042  ...         NaN        3,425.00
47286         NaN        NaN  ...         NaN             NaN
47287         NaN        NaN  ...         NaN             NaN
47288         NaN    Total    ...  106,607.00    9,868,583.13
47289         NaN    Total    ...  936,348.00  115,105,123.71

[47290 rows x 9 columns]
             Date   Voucher  ...        Disc  Voucher Amount
0        1/4/2018   Sal:146  ...         NaN       66,724.00
1        1/4/2018   Sal:146  ...         NaN             NaN
2        1/4/2018   Sal:146  ...         NaN   

## Data Cleaning

In [2]:

data = pd.concat([data_2017, data_2018, data_2019], ignore_index=True, sort=False)
print(data)

data = data.drop(columns=['Gross','Disc','Voucher Amount'],axis=1)
print(data)

data = data[data['Date'] != ' '].dropna(subset=['Date'])
print(data)

data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
print(data['Date'])

data['Voucher'] = data['Voucher'].str.slice(start=4, stop=None, step=1).astype(int)
print(data['Voucher'])

data['Party'] = data['Party'].str.upper()
data['Product'] = data['Product'].str.upper()

print(data['Party'])
print(data['Product'])



              Date   Voucher  ...        Disc Voucher Amount
0         1/4/2017     Sal:1  ...         NaN      13,100.00
1         1/4/2017     Sal:1  ...         NaN            NaN
2         1/4/2017     Sal:2  ...         NaN      30,990.00
3         1/4/2017     Sal:2  ...         NaN            NaN
4         1/4/2017     Sal:2  ...         NaN            NaN
...            ...       ...  ...         ...            ...
111201  10/10/2019  Sal:4935  ...         NaN            NaN
111202         NaN       NaN  ...         NaN            NaN
111203         NaN       NaN  ...         NaN            NaN
111204         NaN   Total    ...   20,680.00   2,189,014.50
111205         NaN   Total    ...  672,984.00  52,830,224.40

[111206 rows x 9 columns]
              Date   Voucher  ...           Qty          Rate
0         1/4/2017     Sal:1  ...             2      1,690.00
1         1/4/2017     Sal:1  ...             6      1,620.00
2         1/4/2017     Sal:2  ...           500        

Convert the Qty column into an integer

In [3]:
# converting the entries with ',' and '.00' to integers only

data['Qty'] = data['Qty'].str.replace(',','').astype(float).astype(int)
print(data['Qty'])

0           2
1           6
2         500
3           6
4           5
         ... 
111197    140
111198    600
111199    320
111200    800
111201    400
Name: Qty, Length: 95562, dtype: int64


Eliminate ',' in the Rate column

In [4]:
data['Rate'] = data['Rate'].str.replace(',','').astype(float)
print(data['Rate'])

0         1690.0
1         1620.0
2           23.0
3         1620.0
4         1690.0
           ...  
111197      26.0
111198       8.4
111199      16.0
111200       8.5
111201      16.0
Name: Rate, Length: 95562, dtype: float64


Sorting the Sales Transaction file in the order of Date and Voucher

In [5]:
data = data.sort_values(['Date','Voucher'])
print(data)

             Date  Voucher  ...  Qty    Rate
0      2017-04-01        1  ...    2  1690.0
1      2017-04-01        1  ...    6  1620.0
2      2017-04-01        2  ...  500    23.0
3      2017-04-01        2  ...    6  1620.0
4      2017-04-01        2  ...    5  1690.0
...           ...      ...  ...  ...     ...
111197 2019-10-10     4935  ...  140    26.0
111198 2019-10-10     4935  ...  600     8.4
111199 2019-10-10     4935  ...  320    16.0
111200 2019-10-10     4935  ...  800     8.5
111201 2019-10-10     4935  ...  400    16.0

[95562 rows x 6 columns]


## Understanding the Data

In [11]:
print(data.shape)
print(data.describe())
print(data.info())

(95562, 6)
            Voucher           Qty           Rate
count  95562.000000  95562.000000   95561.000000
mean    4469.104006    182.157657     249.612310
std     2810.486250    725.256966    2013.475202
min        1.000000      0.000000       0.010000
25%     2047.000000     10.000000      16.000000
50%     4186.500000     50.000000      28.000000
75%     6854.000000    150.000000     140.000000
max    10042.000000  35000.000000  200000.000000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 95562 entries, 0 to 111201
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     95562 non-null  datetime64[ns]
 1   Voucher  95562 non-null  int64         
 2   Party    95562 non-null  object        
 3   Product  95562 non-null  object        
 4   Qty      95562 non-null  int64         
 5   Rate     95561 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 5.1+ M

Top Selling Products :

In [12]:
# Find the no of units sold of each product - to Find the unit price of each product max of price considered
top_selling_items = data.groupby('Product').agg({'Qty':'sum', 'Rate':'max'})
print(top_selling_items)

# Reset the index by converting the Product into a column
top_selling_items.reset_index(inplace=True)

                      Qty    Rate
Product                          
1.25 COOLDRINKS         2   670.0
10" CLASSIFOAM-1200    37  1680.0
10" ESSFOAM LOOSE     310    35.0
10" GREEN               8  1610.0
10" SILVER HEAVY       10    25.0
...                   ...     ...
ZEN-D CHEAP             7   120.0
ZEN-REALPACK           63    90.0
ZEND-1ST             1095   120.0
ZEND-CLASSIC         1181   120.0
ZEND-PREMIUM         1977   100.0

[866 rows x 2 columns]


Rank the product by most Quantity sold

In [14]:
top_selling_items['Top_Sell_Rank'] = top_selling_items['Qty'].rank(method='min',ascending=False).astype(int)

# List the top 20 items sold
top_selling_items.sort_values('Qty', ascending=False).head(10)

,Product,Qty,Rate,Top_Sell_Rank
91,14 GREEN,3474510,400.0,1
282,CYCLE-BLU-10*12,741675,7.1,2
177,500ML PLASTIC BOULS,587600,3.8,3
314,DURGA 10*12 BLUE,530384,110.0,4
43,10*12 SARAS-NAT,433630,81.0,5
279,CYCLE-BK-10*12,431060,13.0,6
842,VISHNU 250ML,400239,35.0,7
638,ROBO 10*12,353880,5.5,8
37,10*12 KRISHNA-BK(10,349720,15.0,9
824,TIRUMALA-50(13*16),346776,28.0,10


**To Ignores the multiple no of orders created in a single day, Consider Date column instead of Voucher in counting the no of orders placed for a product.**

In [15]:
# Remove duplicate records at Product, Date and Party level
unique_items = data.drop_duplicates(['Product','Date','Party'])

# Find the no of orders placed and the unique no of customers placed orders, of each product
popular_items = unique_items.groupby('Product').agg({'Date':'count', 'Party':'nunique'})
popular_items.columns=['No_of_Orders','No_of_Customers']

# Reset the index by converting the Product into a column
popular_items.reset_index(inplace=True)
print(popular_items.head(10))

               Product  No_of_Orders  No_of_Customers
0      1.25 COOLDRINKS             1                1
1  10" CLASSIFOAM-1200            24                8
2    10" ESSFOAM LOOSE             2                2
3            10" GREEN             1                1
4     10" SILVER HEAVY             1                1
5  10" THERMOCOL PRINT             2                2
6        10*10 CITIZEN             9                7
7         10*10 DHAVAT             7                3
8        10*10 JANATHA             1                1
9           10*10 MORE           193               46


# Find Popular Items
### Products with high no of orders can be considered as most frequently purchased items
To find the most popular items we include the number of customers purchased and provide more weightage to products purchased by more customers

### Weighted No of Orders (W) = O * (C / M) 

In [16]:
# No_of_Orders - O
Orders = popular_items['No_of_Orders']

# No_of_Customers purchased the product - C

Customers_Purchased = popular_items['No_of_Customers']

# Maximum no of customers made transactions in the entire period - M
Max_cust = popular_items['No_of_Customers'].max()

popular_items['Weighted_No_of_Orders'] = Orders * (Customers_Purchased / Max_cust)
print(popular_items)

                 Product  No_of_Orders  No_of_Customers  Weighted_No_of_Orders
0        1.25 COOLDRINKS             1                1               0.004098
1    10" CLASSIFOAM-1200            24                8               0.786885
2      10" ESSFOAM LOOSE             2                2               0.016393
3              10" GREEN             1                1               0.004098
4       10" SILVER HEAVY             1                1               0.004098
..                   ...           ...              ...                    ...
861          ZEN-D CHEAP             2                2               0.016393
862         ZEN-REALPACK             6                5               0.122951
863             ZEND-1ST            56               27               6.196721
864         ZEND-CLASSIC            35                9               1.290984
865         ZEND-PREMIUM            64               34               8.918033

[866 rows x 4 columns]


Finding Rank of the product by weighted no of orders

In [17]:
popular_items['Popularity_Rank'] = popular_items['Weighted_No_of_Orders'].rank(method='min',ascending=False).astype(int)

# List of top 20 most popular items sold
popular_items.sort_values('Popularity_Rank',ascending=True).head(10)

,Product,No_of_Orders,No_of_Customers,Weighted_No_of_Orders,Popularity_Rank
91,14 GREEN,2365,244,2365.000000,1
825,TIRUMALA-50(16*20),2123,224,1948.983607,2
238,BLACK DOG-350ML,1945,206,1642.090164,3
223,APPLE WATER,1853,203,1541.635246,4
843,VISHNU 300ML,2060,160,1350.819672,5
842,VISHNU 250ML,1962,166,1334.803279,6
19,10*10 TEJA,1454,184,1096.459016,7
230,BAHUBALI WINE,1468,181,1088.967213,8
824,TIRUMALA-50(13*16),1491,176,1075.475410,9
465,NO-1,1178,195,941.434426,10


### Merge Top Selling Items Rank and Popularity Rank dataframes

In [21]:

product_rankings = pd.merge( top_selling_items, popular_items, how='inner', on='Product')
print(product_rankings)

# Getting only the Product, Price and Rank columns
product_rankings = product_rankings[['Product','Rate','Top_Sell_Rank','Popularity_Rank']]
print(product_rankings)


                 Product   Qty  ...  Weighted_No_of_Orders  Popularity_Rank
0        1.25 COOLDRINKS     2  ...               0.004098              732
1    10" CLASSIFOAM-1200    37  ...               0.786885              416
2      10" ESSFOAM LOOSE   310  ...               0.016393              655
3              10" GREEN     8  ...               0.004098              732
4       10" SILVER HEAVY    10  ...               0.004098              732
..                   ...   ...  ...                    ...              ...
861          ZEN-D CHEAP     7  ...               0.016393              655
862         ZEN-REALPACK    63  ...               0.122951              552
863             ZEND-1ST  1095  ...               6.196721              245
864         ZEND-CLASSIC  1181  ...               1.290984              366
865         ZEND-PREMIUM  1977  ...               8.918033              219

[866 rows x 8 columns]
                 Product    Rate  Top_Sell_Rank  Popularity_Rank

List the Product Rankings

In [23]:
product_rankings.sort_values('Popularity_Rank',ascending=True).head(10)

,Product,Rate,Top_Sell_Rank,Popularity_Rank
91,14 GREEN,400.0,1,1
825,TIRUMALA-50(16*20),30.0,19,2
238,BLACK DOG-350ML,33.0,15,3
223,APPLE WATER,29.0,11,4
843,VISHNU 300ML,50.0,16,5
842,VISHNU 250ML,35.0,7,6
19,10*10 TEJA,142.0,64,7
230,BAHUBALI WINE,31.0,20,8
824,TIRUMALA-50(13*16),28.0,10,9
465,NO-1,160.0,123,10


### Store Preprocessed Data to csv

In [25]:
product_rankings.to_csv('Product_Rankings.csv', index=False)

### Creating a Pickle file and storing with the Ranking dataframe

In [24]:
import pickle
pickle.dump(product_rankings, open('prod_ranking_model.pkl','wb'))